深度学习 Deep Learning
=============

任务2 Assignment 2
------------
以前在 1_notmnist. ipynb 中，我们创造了一个带有格式的数据集，用于 notmnist 数据集的培训、开发和测试。

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

该任务的目标是逐步使用TensorFlow来训练更深入和更准确的模型。

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [10]:
# 导入所有依赖包
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

首先读取从1_notmnist产生的数据

First reload the data we generated in `1_notmnist.ipynb`.

In [11]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


重新格式化为一个更适合我们要训练的模型的形状

Reformat into a shape that's more adapted to the models we're going to train:
- 数据是一个矩阵
- 标签是一个独热编码
- data as a flat matrix,
- labels as float one-hot encodings.

In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


我们首先使用简单梯度下降训练多项Logistic回归。

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow的工作原理如下：
* 首先，描述要执行的计算：输入，变量和操作的情况。通过创建节点来计算图形。此描述全部包含在以下内容中：

* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...
* 然后，您可以通过调用sessi.run()在这个图上运行这些操作。从返回的图中获取输出。运行时的操作全部包含在下面的块中

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...
          
让我们将所有数据加载到 TensorFlow 中, 并构建与我们的训练相对应的计算图:

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [13]:
# 即使用梯度下降训练, 如此多的数据也是难以承受的
# With gradient descent training, even this much data is prohibitive.
# 分组训练，加快周转时间
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # 导入数据.
    # 通过加载读取训练，验证和测试数据创建常量张量
    # Load the training, validation and test data into constants that are
    # 附加到图形
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 权重变量.
    # 这些是我们将要训练的参数。
    # These are the parameters that we are going to be training. 
    # 将使用（截断）正态分布后的随机值来初始化权重矩阵，偏差初始化为零
    # The weight matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # 训练
    # Training computation.
    # 我们将把输入和权重方程相乘，再加上偏差。
    # We multiply the inputs with the weight matrix, and add biases. 
    # 我们计算softmax和交叉熵（它是TensorFlow中的一个操作，因为它是很常见的，可以优化）
    # We compute the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). 
    # 我们把这个熵的平均值贯穿于所有训练示例中: 这是我们的损失。
    # We take the average of this cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # 优化.我们将使用梯度下降找到最小损失
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # 根据训练，验证和测试数据预测
    # Predictions for the training, validation, and test data.
    # 这不是训练的一部分，但是在这里，我们可以看到训练时报告的精准性
    # These are not part of training, 
    # but merely here so that we can report accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

让我们运行这个计算并迭代:

Let's run this computation and iterate:

In [14]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # 这是一个一次性操作，可以确保参数按照我们在图中描述的初始化：矩阵的随机权重，偏差为零。
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # 运行计算。我们用.run()运行优化
    # Run the computations. We tell .run() that we want to run the optimizer,
    # 获取损失值和训练预测返回到numpy数组
    # and get the loss value and the training predictions returned as numpy arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
        # 调用 valid_prediction.eval() 基本上就像运行.run(),但是只得到一个numpy数组
        # Calling .eval() on valid_prediction is basically like calling run(), 
        # but just to get that one numpy array. 
        # 注意，他重新计算了图的依赖
        # Note that it recomputes all its graph dependencies.
        print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.883345
Training accuracy: 9.8%
Validation accuracy: 11.6%
Loss at step 100: 2.362335
Training accuracy: 71.5%
Validation accuracy: 70.4%
Loss at step 200: 1.872376
Training accuracy: 74.7%
Validation accuracy: 72.8%
Loss at step 300: 1.609457
Training accuracy: 76.4%
Validation accuracy: 73.7%
Loss at step 400: 1.437497
Training accuracy: 77.4%
Validation accuracy: 74.2%
Loss at step 500: 1.313284
Training accuracy: 78.2%
Validation accuracy: 74.3%
Loss at step 600: 1.217437
Training accuracy: 78.7%
Validation accuracy: 74.8%
Loss at step 700: 1.140306
Training accuracy: 79.3%
Validation accuracy: 74.8%
Loss at step 800: 1.076317
Training accuracy: 79.6%
Validation accuracy: 75.1%
Test accuracy: 82.0%


现在我们来改用随机梯度下降训练，这样快很多。
Let's now switch to stochastic gradient descent training instead, which is much faster.

图是类似的，除了将所有训练数据保存到一个常量节点中，我们创建一个占位符（Placeholder）节点, 它将在每次调用session.run（）时提供实际数据。

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [15]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # 输入数据。对于培训数据，我们使用一个占位符在运行时使用一个培训小批处理
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 变量.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # 训练计算.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # 优化.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # 训练，验证和测试数据的预测
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

让我们运行它

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # 在训练数据中选择一个偏移量，该数据已被随机化。
    # Pick an offset within the training data, which has been randomized.
    # 提醒: 我们能使用更好的随机化穿过所有数据
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # 生成一个迷你批次
    # Generate a mini batch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # 准备一个字典，告诉会话在哪里提供迷你批处理
    # Prepare a dictionary telling the session where to feed the minibatch.
    # 字典的关键是要馈送的图标的占位符节点和值是numpy数组馈送
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.143641
Minibatch accuracy: 10.2%
Validation accuracy: 13.2%
Minibatch loss at step 500: 1.482917
Minibatch accuracy: 80.5%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.124429
Minibatch accuracy: 77.3%
Validation accuracy: 76.5%
Minibatch loss at step 1500: 0.855406
Minibatch accuracy: 83.6%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 0.687463
Minibatch accuracy: 83.6%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 0.982325
Minibatch accuracy: 77.3%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 1.132665
Minibatch accuracy: 78.1%
Validation accuracy: 79.1%
Test accuracy: 86.7%


---
问题 Problem
-------
转换这个逻辑回归用例变成一个隐藏层的神经网络用整数线性单位nn.rule和1024个隐藏节点.这个模型应该改善你的训练和测试精准性

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [45]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # 输入数据。对于培训数据，我们使用一个占位符在运行时使用一个培训小批处理
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 变量.
    # 第一层
    # truncated_normal 从一个正态分布片段中输出随机数值,
    # 生成的值会遵循一个指定了平均值和标准差的正态分布，只保留两个标准差以内的值，超出的值会被弃掉重新生成。
    # 返回 一个指定形状并用正态分布片段的随机值填充的张量
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))

    # 第二层
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))


    
  
  
    # 训练计算.
    # logits = tf.matmul(tf_train_dataset, weights) + biases
    train_logits = tf.add(tf.matmul(tf_train_dataset, weights1),biases1)
    train_logits = tf.nn.relu(train_logits)
    train_logits = tf.add(tf.matmul(train_logits,weights2),biases2)

    valid_logits = tf.add(tf.matmul(tf_valid_dataset, weights1),biases1)
    valid_logits = tf.nn.relu(valid_logits)
    valid_logits = tf.add(tf.matmul(valid_logits,weights2),biases2)

    test_logits = tf.add(tf.matmul(tf_test_dataset, weights1),biases1)
    test_logits = tf.nn.relu(test_logits)
    test_logits = tf.add(tf.matmul(test_logits,weights2),biases2)
    


    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=train_logits, labels=tf_train_labels))
    # 优化.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # 训练，验证和测试数据的预测
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)



In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # 在训练数据中选择一个偏移量，该数据已被随机化。
    # Pick an offset within the training data, which has been randomized.
    # 提醒: 我们能使用更好的随机化穿过所有数据
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # 生成一个迷你批次
    # Generate a mini batch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # 准备一个字典，告诉会话在哪里提供迷你批处理
    # Prepare a dictionary telling the session where to feed the minibatch.
    # 字典的关键是要馈送的图标的占位符节点和值是numpy数组馈送
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 332.016418
Minibatch accuracy: 11.7%
Validation accuracy: 22.6%
Minibatch loss at step 500: 18.727076
Minibatch accuracy: 82.0%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 14.503419
Minibatch accuracy: 78.1%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 5.280657
Minibatch accuracy: 86.7%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 2.402635
Minibatch accuracy: 86.7%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 3.886218
Minibatch accuracy: 79.7%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 2.336560
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Test accuracy: 89.1%


In [40]:
batch_size = 128
hiden_layer_node_num = 1024

# tensorflow作为一个基于图结构的深度学习框架，内部通过session实现图和计算内核的交互
# tensorflow因为代码执行过程是先构建图，然后在执行，所以对中间过程的调试不太方便，所以提供了一个tensorboard工具来便于调试
graph = tf.Graph()
with graph.as_default():
    # -----------------------------------------1
    # 输入 
    # placeholder 插入一个待初始化的张量占位符
    # 重要事项：这个张量被求值时会产生错误。 
    # 它的值必须在Session.run(), Tensor.eval() 或 Operation.run() 中使用feed_dict的这个可选参数来填充。
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # 创建一个常量张量
    # tf_valid_dataset = Tensor("Const:0", shape=(10000, 784), dtype=float32)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # ------------------------------------------2
    # 变量
    # 当你训练一个模型的时候，你使用变量去保存和更新参数。
    # 在Tensorflow中变量是内存缓冲区中保存的张量（tensor）。
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hiden_layer_node_num]))
    biases1 = tf.Variable(tf.zeros([hiden_layer_node_num]))
    
    # 输入层输出 (批量尺寸batch_size, 隐藏节点hiden_layer_node_num)
    weights2 = tf.Variable(tf.truncated_normal([hiden_layer_node_num, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # ------------------------------------------3
    # 训练计算.
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
    # 可跨越维度的计算张量各元素的平均值
    # 计算loss是代价值，也就是我们要最小化的值
    # 第一个参数logits：就是神经网络最后一层的输出，
    # 如果有batch的话，它的大小就是[batchsize，num_classes]，单样本的话，大小就是num_classes
    # 第二个参数labels：实际的标签，大小同上
    # 第一步是先对网络最后一层的输出做一个softmax，这一步通常是求取输出属于某一类的概率
    # 第二步是softmax的输出向量[Y1，Y2,Y3...]和样本的实际标签做一个交叉熵
    # 注意！！！这个函数的返回值并不是一个数，而是一个向量.
    # 如果要求交叉熵，我们要再做一步tf.reduce_sum操作,就是对向量里面所有元素求和，最后才得到. 
    # 如果求loss，则要做一步tf.reduce_mean操作，对向量求均值！
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  
    # -------------------------------------------4
    # 最优化.因为深度学习常见的是对于梯度的优化，也就是说，优化器最后其实就是各种对于梯度下降算法的优化。 
    # 函数training()通过梯度下降法为最小化损失函数增加了相关的优化操作，在训练过程中，
    # 先实例化一个优化函数，比如 tf.train.GradientDescentOptimizer，并基于一定的学习率进行梯度优化训练：
    # learning_rate参数：要使用的学习率 
    # minimize：非常常用的一个函数 通过更新var_list来减小loss，这个函数就是前面compute_gradients() 和apply_gradients().的结合
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    
    #--------------------------------------------5
    # 预测训练, 验证,和测试数据.
    # tf.nn.softmax仅产生将softmax function应用于输入张量的结果。 
    # softmax 压扁输入，使sum(输入)= 1;这是一种正常化的方式。 
    # softmax的输出形状与输入相同 – 它只是对值进行归一化。 
    # softmax的输出可以解释为概率。
    # a = tf.constant(np.array([[.1, .3, .5, .9]]))
    # print s.run(tf.nn.softmax(a))
    # [[ 0.16838508  0.205666    0.25120102  0.37474789]]
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

    
num_steps = 3001
# Session（会话）是负责graph（图）执行的。
with tf.Session(graph=graph) as session:
    # 准备计算所需要初始化的变量，但是返回结果将会是‘None’
    tf.initialize_all_variables().run()
    print("Initialized")
    # 设定输出日志路径和图
    summary_writer = tf.summary.FileWriter('uda_logs', session.graph)
    
    for step in range(num_steps):
        # 在训练数据中选择一个已被随机化的偏移量.
        # 提醒: 我们能使用更好的随机化穿过所有数据.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # 生成一个小批量数据
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # feed_dict的作用是给使用placeholder创建出来的tensor赋值。
        # 其实，他的作用更加广泛：feed 使用一个 值临时替换一个 op 的输出结果. 
        # 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失.
        #  传递值到tf的命名空间
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summary, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        # 输出日志
        merged = tf.summary.merge_all()
        summary_writer.add_summary(summary, step)
        if (step % 500 == 0):
            # print(type(l))
            # print(type(predictions))
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 346.990967
Minibatch accuracy: 8.6%
Validation accuracy: 33.2%
Minibatch loss at step 500: 25.715347
Minibatch accuracy: 80.5%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 8.652662
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 4.688735
Minibatch accuracy: 85.9%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 3.423947
Minibatch accuracy: 86.7%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 6.644867
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 2.125021
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Test accuracy: 89.5%
